### Import (라이브러리)

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

import os
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from tqdm import tqdm

### Data Load (Train, Test 데이터 로드)

In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
train

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,...,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부
0,TRZKPL,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0
1,TRYBLT,만45-50세,NaN,IVF,ICSI,0,알 수 없음,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
2,TRVNRY,만18-34세,NaN,IVF,IVF,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,2.0,NaN,0
3,TRJXFG,만35-37세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
4,TRVNRY,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,TRYBLT,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,5.0,NaN,0
256347,TRYBLT,만38-39세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,1
256348,TRVNRY,만35-37세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0
256349,TRZKPL,만38-39세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,1.0,NaN,1


In [ ]:
# Train Data X, y 분리

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# y

In [4]:
test = pd.read_csv('./test.csv').drop(columns=['ID'])
test

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,...,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일
0,TRYBLT,만35-37세,NaN,IVF,IVF,1,기록되지 않은 시행,0.0,NaN,0.0,...,1.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN
1,TRDQAZ,만18-34세,NaN,IVF,IVF,1,기록되지 않은 시행,0.0,NaN,0.0,...,1.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN
2,TRCMWS,만40-42세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,1.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN
3,TRJXFG,만40-42세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,1.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,2.0,NaN
4,TRJXFG,만35-37세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,0.0,...,1.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90062,TRDQAZ,만18-34세,NaN,IVF,ICSI:ICSI,1,기록되지 않은 시행,0.0,NaN,1.0,...,1.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN
90063,TRYBLT,만43-44세,NaN,IVF,Unknown,0,알 수 없음,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
90064,TRVNRY,만18-34세,NaN,IVF,IVF,0,알 수 없음,1.0,NaN,0.0,...,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,5.0,NaN
90065,TRCMWS,만43-44세,NaN,IVF,Unknown,0,알 수 없음,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4.0,0.0


### Data Pre-processing (데이터 전처리)

In [5]:
# 결측치 비율이 50%가 넘는 칼럼 제거

def highly_null(df, threshold=0.5):
    df_copy = df.copy()
    missing_ratio = df_copy.isnull().mean()
    null_culumns = df_copy.columns[missing_ratio > threshold]
    df_copy.drop(columns=null_culumns, inplace=True)
    return df_copy

X = highly_null(X)
test = highly_null(test)

X
# test

,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,정자 출처,난자 기증자 나이,정자 기증자 나이,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,난자 채취 경과일,난자 혼합 경과일,배아 이식 경과일
0,TRZKPL,만18-34세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,3.0
1,TRYBLT,만45-50세,IVF,ICSI,0,알 수 없음,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2,TRVNRY,만18-34세,IVF,IVF,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,2.0
3,TRJXFG,만35-37세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,NaN
4,TRVNRY,만18-34세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,TRYBLT,만18-34세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,5.0
256347,TRYBLT,만38-39세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,3.0
256348,TRVNRY,만35-37세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,3.0
256349,TRZKPL,만38-39세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [6]:
train_columns = X.columns
test_columns = test.columns
columns_in_both = set(train_columns).intersection(set(test_columns))
print(f"same columns: {len(columns_in_both)}")
print(f"train columns: {len(train_columns)}")
print(f"test columns: {len(test_columns)}")

same columns: 61
train columns: 61
test columns: 61


In [7]:
# 같은 데이터로만 구성된 칼럼 제거

def remove_all_same(df):
    df_copy = df.copy()
    same_value_columns = df_copy.columns[df_copy.nunique()==1]
    df_copy.drop(columns = same_value_columns, inplace = True)
    return df_copy

X = remove_all_same(X)
test = remove_all_same(test)

X
# test

,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,난자 출처,정자 출처,난자 기증자 나이,정자 기증자 나이,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,난자 혼합 경과일,배아 이식 경과일
0,TRZKPL,만18-34세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,3.0
1,TRYBLT,만45-50세,IVF,ICSI,0,알 수 없음,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,NaN
2,TRVNRY,만18-34세,IVF,IVF,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,2.0
3,TRJXFG,만35-37세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,NaN
4,TRVNRY,만18-34세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,TRYBLT,만18-34세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,5.0
256347,TRYBLT,만38-39세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,3.0
256348,TRVNRY,만35-37세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,3.0
256349,TRZKPL,만38-39세,IVF,ICSI,1,기록되지 않은 시행,0.0,0.0,0,0,...,본인 제공,배우자 제공,알 수 없음,알 수 없음,0.0,1.0,0.0,0.0,0.0,1.0


In [8]:
train_columns = X.columns
test_columns = test.columns
columns_in_both = set(train_columns).intersection(set(test_columns))
print(f"same columns: {len(columns_in_both)}")
print(f"train columns: {len(train_columns)}")
print(f"test columns: {len(test_columns)}")

same columns: 59
train columns: 59
test columns: 59


In [24]:
# null 개수 확인

print(f"train null : {X.isnull().sum().sum()}")
print(f"test null : {test.isnull().sum().sum()}")      

train null : 185375
test null : 64289


In [17]:
# 범주형

categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    # "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    # "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    # "PGD 시술 여부",
    # "PGS 시술 여부"
]

In [18]:
# 카테고리(범주)형 컬럼들을 문자열로 변환

for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [19]:
# 문자열을 수치형으로 변환

# 학습 데이터에 없는 새로운 카테고리 값 처리 방법 지정
# 알 수 없는 값은 -1로 인코딩
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 학습 데이터 변환
# 카테고리 값들을 학습하고 변환
X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

# 테스트 데이터 변환
# 학습된 인코딩 규칙으로 테스트 데이터 변환
X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

X_train_encoded
# X_test_encoded

,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,난자 출처,정자 출처,난자 기증자 나이,정자 기증자 나이,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,난자 혼합 경과일,배아 이식 경과일
0,6.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
1,5.0,5.0,1.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,NaN
2,3.0,0.0,1.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,2.0
3,2.0,1.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,NaN
4,3.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,5.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,5.0
256347,5.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
256348,3.0,1.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
256349,6.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,1.0


In [22]:
# 원래 수치형이었던 칼럼들

numeric_columns = [
    # "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    # "난자 채취 경과일",
    # "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    # "배아 해동 경과일"
]

In [23]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

X_train_encoded
# X_test_encoded

,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,난자 출처,정자 출처,난자 기증자 나이,정자 기증자 나이,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,난자 혼합 경과일,배아 이식 경과일
0,6.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
1,5.0,5.0,1.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3.0,0.0,1.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,2.0
3,2.0,1.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,5.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,5.0
256347,5.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
256348,3.0,1.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,3.0
256349,6.0,2.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,4.0,6.0,0.0,1.0,0.0,0.0,0.0,1.0


### Train (모델 학습)

In [26]:
# 평가지표

def get_clf_eval(y_test, pred=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [25]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

### Predict

In [31]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

for i in pred_proba:
    print(i)

0.01
0.01
0.07
0.05
0.43
0.04
0.24
0.23
0.17
0.0
0.07
0.36
0.38
0.46
0.22
0.0
0.28
0.6
0.45
0.61
0.15
0.32
0.4
0.17
0.14
0.0
0.02
0.67
0.17
0.06
0.75
0.08
0.13
0.08
0.5
0.0
0.33
0.24
0.31
0.0
0.22
0.65
0.12
0.68
0.32
0.64
0.65
0.38
0.0
0.6
0.13
0.47
0.04
0.42
0.0
0.44
0.0
0.18
0.16
0.17
0.01
0.05
0.25
0.15
0.3
0.13
0.18
0.2
0.23
0.4
0.57
0.09
0.12
0.15
0.02
0.62
0.0
0.02
0.23
0.1
0.29
0.0
0.09
0.08
0.62
0.27
0.0
0.5
0.1
0.26
0.39
0.01
0.27
0.47
0.0
0.0
0.42
0.13
0.37
0.345
0.08
0.36
0.13
0.48
0.21
0.46
0.38
0.0
0.36
0.0
0.3
0.25
0.0
0.02
0.06
0.43
0.02
0.36
0.51
0.22
0.0
0.0
0.5
0.55
0.23
0.11
0.0
0.08
0.05
0.13
0.26
0.88
0.0
0.06
0.53
0.0
0.27
0.02
0.03
0.2
0.0
0.0
0.32
0.0
0.15
0.37
0.09
0.47
0.18
0.42
0.0
0.01
0.23
0.12
0.04
0.35
0.57
0.24
0.56
0.58
0.74
0.72
0.21
0.0
0.32
0.16
0.46
0.53
0.15
0.45
0.21
0.17
0.17
0.41
0.1
0.25
0.38
0.24
0.3
0.06
0.76
0.64
0.33
0.39
0.02
0.09
0.59
0.16
0.17
0.28
0.56
0.23
0.2
0.09
0.77
0.59
0.26
0.35
0.08
0.32
0.57
0.46
0.38
0.28
0.0
0.25
0.0
0.17
0.0

In [27]:
get_clf_eval(y, model.predict(X_train_encoded))

오차 행렬
[[190122      1]
 [    53  66175]]
정확도: 0.9998, 정밀도: 1.0000, 재현율: 0.9992,    F1: 0.9996


### Submission

In [33]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [34]:
sample_submission.to_csv('./baseline_submit.csv', index=False)